In [ ]:
from model import modello_albero
import pyomo.environ as pyo

### PREPROCESS DATASET ###


In [ ]:
###  TRAINING MODELLO ###
modello = modello_albero()

opt = pyo.SolverFactory('gurobi')
results = opt.solve(modello) # solves and updates model
pyo.assert_optimal_termination(results)

In [ ]:
### VALIDAZIONE ###

